In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('anzahl-sbb-bahnhofbenutzer.csv')
data_pop = pd.read_excel('data_pop.xlsx')
data_red = pd.read_excel('data_pop_modif.xlsx')

In [3]:
# Dividi la colonna in più colonne usando ';' come separatore
data_split = data['Bahnhof_Gare_Stazione;Unité;Jahr;Anzahl Bahnhofbenutzer'].str.split(';', expand=True)

# Assegna nomi alle colonne
data_split.columns = ['Bahnhof', 'Unité', 'Jahr', 'Anzahl Bahnhofbenutzer']

In [4]:
data_2024 = data_split[(data_split['Unité']== 'DP/jour ouvré') & (data_split['Jahr']=='2024')]
origin = data_2024.drop(['Unité', 'Jahr'], axis=1)
data_2024

,Bahnhof,Unité,Jahr,Anzahl Bahnhofbenutzer
67,Neuchâtel,DP/jour ouvré,2024,28900
77,Thun,DP/jour ouvré,2024,40400
83,Zürich Altstetten,DP/jour ouvré,2024,69700
91,Zürich Hardbrücke,DP/jour ouvré,2024,50700
98,Bellinzona,DP/jour ouvré,2024,16100
100,Lugano,DP/jour ouvré,2024,34100
111,Genève-Eaux-Vives,DP/jour ouvré,2024,14500
134,St. Gallen,DP/jour ouvré,2024,77400
140,Winterthur,DP/jour ouvré,2024,134700
150,Zug,DP/jour ouvré,2024,58900


In [5]:
# List of stations to remove
to_remove = ['Uster']

# Filter rows where 'Station_Name' is NOT in to_remove
filtered_df = origin[~origin['Bahnhof'].isin(to_remove)]

# Optional: Reset index if needed
filtered_df = filtered_df.reset_index(drop=True)


In [6]:

data_2024.rename(columns={'Anzahl Bahnhofbenutzer': 'origin'}, inplace=True)   # Rename a column by providing old and new names
data_2024.rename(columns={'Bahnhof': 'city'}, inplace=True) 

data_2024['origin'] = pd.to_numeric(data_2024['origin'], errors='coerce')

data_2024 = data_2024.drop(['Unité', 'Jahr'], axis=1)




C:\Users\liuzz_n67s76z\AppData\Local\Temp\ipykernel_14944\4129284884.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_2024.rename(columns={'Anzahl Bahnhofbenutzer': 'origin'}, inplace=True)   # Rename a column by providing old and new names
C:\Users\liuzz_n67s76z\AppData\Local\Temp\ipykernel_14944\4129284884.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_2024.rename(columns={'Bahnhof': 'city'}, inplace=True)
C:\Users\liuzz_n67s76z\AppData\Local\Temp\ipykernel_14944\4129284884.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

In [7]:
df = data_2024

# 1. Lista delle città da unire
cities_to_merge_zh = ["Zürich HB",'Zürich Enge','Zürich Hardbrücke', 'Zürich Altstetten' ]
cities_to_merge_ge = ['Genève', 'Genève-Eaux-Vives']

# 2. Filtra e somma
mask_zh = df["city"].isin(cities_to_merge_zh)
mask_ge = df["city"].isin(cities_to_merge_ge)

merged_origin_zh = df.loc[mask_zh, "origin"].sum()
merged_origin_ge = df.loc[mask_ge, "origin"].sum()


# 3. Nuova riga unificata
new_row_zh = pd.DataFrame({"city": ["Zürich HB"], "origin": [merged_origin_zh]})
new_row_ge = pd.DataFrame({"city": ["Genève"], "origin": [merged_origin_ge]})


# 4. Filtra il DataFrame originale e concatena
df_filtered = df[~mask_zh]
df_filtered = df_filtered[~mask_ge]
df_filtered_zh = pd.concat([df_filtered, new_row_zh], ignore_index=True)
df_final = pd.concat([df_filtered_zh, new_row_ge], ignore_index=True)

C:\Users\liuzz_n67s76z\AppData\Local\Temp\ipykernel_14944\4269005010.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_filtered = df_filtered[~mask_ge]


In [8]:
# List of stations to remove
to_remove = ['Uster', 'Zürich Stadelhofen']

# Filter rows where 'Station_Name' is NOT in to_remove
df_final = df_final[~df_final['city'].isin(to_remove)]

# Optional: Reset index if needed
df_final = df_final.reset_index(drop=True)
df_final

,city,origin
0,Neuchâtel,28900
1,Thun,40400
2,Bellinzona,16100
3,Lugano,34100
4,St. Gallen,77400
5,Winterthur,134700
6,Zug,58900
7,Bern,298900
8,Luzern,145400
9,Baden,55400


In [9]:
total_origin = df_final['origin'].sum()


In [10]:
geneve_station = ['Genève', 'Genève-Aéroport']
production_ge = df_final[df_final['city'].isin(geneve_station)]
total_origin_ge = production_ge['origin'].sum()
production_ge['weight'] = production_ge['origin']/total_origin_ge
production_ge

C:\Users\liuzz_n67s76z\AppData\Local\Temp\ipykernel_14944\3712010864.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  production_ge['weight'] = production_ge['origin']/total_origin_ge


,city,origin,weight
15,Genève-Aéroport,43000,0.18647
20,Genève,187600,0.81353


In [11]:
zurich_station = ['Zürich Oerlikon', 'Zürich HB']
production_zh = df_final[df_final['city'].isin(zurich_station)]
total_origin_zh = production_zh['origin'].sum()

production_zh['weight'] = production_zh['origin']/total_origin_zh
production_zh

C:\Users\liuzz_n67s76z\AppData\Local\Temp\ipykernel_14944\1409896073.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  production_zh['weight'] = production_zh['origin']/total_origin_zh


,city,origin,weight
11,Zürich Oerlikon,113800,0.171437
19,Zürich HB,550000,0.828563


In [12]:
data_red

,city,population,origin
0,Thun,43670,40400
1,Basel SBB,173552,140900
2,Lausanne,141418,127900
3,Winterthur,116906,134700
4,St. Gallen,76931,77400
5,Luzern,83840,145400
6,Lugano,62464,34100
7,Baden,32566,55400
8,Fribourg/Freiburg,37653,31500
9,Biel/Bienne,55070,65200


In [13]:
popolation_total = data_pop['habitant'].sum()

In [14]:
data_pop['weight'] = data_pop['habitant']/(popolation_total)

In [15]:
data_pop

,City,habitant,weight
0,Zürich,427721,0.237127
1,Genève,203840,0.113008
2,Thun,43670,0.024210
3,Basel SBB,173552,0.096216
4,Lausanne,141418,0.078402
5,Winterthur,116906,0.064812
6,St. Gallen,76931,0.042650
7,Luzern,83840,0.046481
8,Lugano,62464,0.034630
9,Baden,32566,0.018054


In [16]:
weight_zh = data_pop[data_pop['City']=='Zürich']

weight_ge = data_pop[data_pop['City']=='Genève']



In [17]:
production_zh['weight_population'] = production_zh['weight']*weight_zh['weight'].values
production_zh['destination'] = production_zh['weight_population']*total_origin

production_zh

C:\Users\liuzz_n67s76z\AppData\Local\Temp\ipykernel_14944\2980270372.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  production_zh['weight_population'] = production_zh['weight']*weight_zh['weight'].values
C:\Users\liuzz_n67s76z\AppData\Local\Temp\ipykernel_14944\2980270372.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  production_zh['destination'] = production_zh['weight_population']*total_origin


,city,origin,weight,weight_population,destination
11,Zürich Oerlikon,113800,0.171437,0.040652,95069.527707
19,Zürich HB,550000,0.828563,0.196474,459474.870291


In [18]:
production_ge['weight_population'] = production_ge['weight']*weight_ge['weight'].values
production_ge['destination'] = production_ge['weight_population']*total_origin
production_ge


C:\Users\liuzz_n67s76z\AppData\Local\Temp\ipykernel_14944\3626330449.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  production_ge['weight_population'] = production_ge['weight']*weight_ge['weight'].values
C:\Users\liuzz_n67s76z\AppData\Local\Temp\ipykernel_14944\3626330449.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  production_ge['destination'] = production_ge['weight_population']*total_origin


,city,origin,weight,weight_population,destination
15,Genève-Aéroport,43000,0.18647,0.021073,49280.409975
20,Genève,187600,0.81353,0.091935,215000.114217


In [19]:
data_red['weight'] = data_red['population']/(popolation_total)

data_red['destination'] = data_red['weight']*total_origin
data_red

,city,population,origin,weight,destination
0,Thun,43670,40400,0.024210,56618.575802
1,Basel SBB,173552,140900,0.096216,225011.840338
2,Lausanne,141418,127900,0.078402,183349.799697
3,Winterthur,116906,134700,0.064812,151569.755500
4,St. Gallen,76931,77400,0.042650,99741.782803
5,Luzern,83840,145400,0.046481,108699.367878
6,Lugano,62464,34100,0.034630,80985.177900
7,Baden,32566,55400,0.018054,42222.132804
8,Fribourg/Freiburg,37653,31500,0.020875,48817.477322
9,Biel/Bienne,55070,65200,0.030531,71398.785652


In [20]:
data_red = data_red.drop(['population','weight'], axis=1)
production_ge = production_ge.drop(['weight', 'weight_population'], axis=1)
production_zh = production_zh.drop(['weight', 'weight_population'], axis=1)

production = pd.concat([production_ge, production_zh], axis=0, ignore_index=True)  # Ignore original index and create a new one

In [21]:
od_df = pd.concat([production, data_red], axis=0, ignore_index=True)  # Ignore original index and create a new one


In [22]:
origin = od_df['origin'].sum()
destination = od_df['destination'].sum()
print(origin, destination)

2338600 2338599.9999999995


In [23]:
od_df[['origin', 'destination']] = np.round(od_df[['origin', 'destination']]).astype(int)


In [24]:
od_df

,city,origin,destination
0,Genève-Aéroport,43000,49280
1,Genève,187600,215000
2,Zürich Oerlikon,113800,95070
3,Zürich HB,550000,459475
4,Thun,40400,56619
5,Basel SBB,140900,225012
6,Lausanne,127900,183350
7,Winterthur,134700,151570
8,St. Gallen,77400,99742
9,Luzern,145400,108699


In [25]:
#od_df.iloc[1, 2] -=1 

In [26]:
origin = od_df['origin'].sum()
destination = od_df['destination'].sum()
print(origin, destination)

2338600 2338600


In [27]:
od_df

,city,origin,destination
0,Genève-Aéroport,43000,49280
1,Genève,187600,215000
2,Zürich Oerlikon,113800,95070
3,Zürich HB,550000,459475
4,Thun,40400,56619
5,Basel SBB,140900,225012
6,Lausanne,127900,183350
7,Winterthur,134700,151570
8,St. Gallen,77400,99742
9,Luzern,145400,108699


In [28]:
od_df.to_csv('prod_att.csv', index=False)                # To CSV

In [29]:
prod_att = pd.read_csv('prod_att.csv')
prod_att

,city,origin,destination
0,Genève-Aéroport,43000,49280
1,Genève,187600,215000
2,Zürich Oerlikon,113800,95070
3,Zürich HB,550000,459475
4,Thun,40400,56619
5,Basel SBB,140900,225012
6,Lausanne,127900,183350
7,Winterthur,134700,151570
8,St. Gallen,77400,99742
9,Luzern,145400,108699


In [30]:
origin = prod_att['origin'].sum()
destination = prod_att['destination'].sum()
print(origin, destination)

2338600 2338600
